In [26]:
import nbformat
import os
from nbconvert.preprocessors import ExecutePreprocessor

In [27]:
# Paths to look for .ipynb
# DIRS = ['../../pandas-basic/teilnehmer/loesung/',]
DIRS = ['../../course-python-beginner/lessons/python introduction']
OUT_DIRS = [] # empty list is same dir default to clean folder
OUT_DIR = 'clean'
POSTFIX = '' # default not postfix
REPLACE_FILENAME = ('', '')

In [28]:
# Options
LAST_ROW_STANDING = True # add a emtpy code cell at the end of the document 
SPECIFIC_VERSION = 1.1

In [29]:
# Functions
def open_nb(filepath):
    with open(filepath, encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)
        
    return nb

def save_nb(d, filepath, nb):
    # check if dir exist
    if not os.path.isdir(os.path.dirname(filepath) + '/clean'):
        os.mkdir(os.path.dirname(filepath) + '/clean')
    
    # create basename
    filename = os.path.basename(filepath).replace(REPLACE_FILENAME[0], REPLACE_FILENAME[1])
    filename = filename.replace('.ipynb', POSTFIX + '.ipynb')
    
    # my files have a ' - Loesung' at the end, which is to remove
    with open(os.path.dirname(filepath) + '/clean/' + filename, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

def add_version(nb):
    nb.metadata['ht_cell_export_version'] = SPECIFIC_VERSION
        
def process_cells(nb):
    out_cells = []
    nb_cell_state = {'last_codecell_delete': False,
                    'last_markdown_delete':False}
    
    # main loop
    for c in nb.get('cells'):
    
        # handle code cells
        if c.get('cell_type') == 'code':

            ht_protected = c.get('metadata').get('ht_protected', None)
            nb_cell_state['last_markdown_delete'] = False
            
            if ht_protected:
                out_cells.append(c)
                nb_cell_state['last_codecell_delete'] = True
            elif ht_protected == False:
                out_cells.append(c)
                nb_cell_state['last_codecell_delete'] = False
            else:
                # kill code cell
                nb_cell_state['last_codecell_delete'] = True

        elif c.get('cell_type') == 'markdown':
            
            ht_enforce_remove = c.get('metadata').get('ht_enforce_remove', None)
            
            # check if last one was markdown enforced or codecell
            if nb_cell_state['last_codecell_delete']:
                out_cells.append(nbformat.v4.new_code_cell())
            if nb_cell_state['last_markdown_delete'] and \
            ht_enforce_remove is None:
                out_cells.append(nbformat.v4.new_markdown_cell())
            
            # reset code cell status
            nb_cell_state['last_codecell_delete'] = False

            if ht_enforce_remove:
                # kill markdown cell
                nb_cell_state['last_markdown_delete'] = True
            elif ht_enforce_remove == False:
                # kill markdown cell
                nb_cell_state['last_markdown_delete'] = False
            else:
                nb_cell_state['last_markdown_delete'] = False
                out_cells.append(c)

    if LAST_ROW_STANDING and nb_cell_state['last_codecell_delete']:
        out_cells.append(nbformat.v4.new_code_cell())
    
    nb['cells'] = out_cells
    return nb

In [30]:
if len(OUT_DIRS) == 1:
    out_dirs = [OUT_DIRS[0]] * len(DIRS)
elif len(OUT_DIRS) == len(DIRS):
    out_dirs = OUT_DIRS
elif len(OUT_DIRS) == 0:
    out_dirs = [d + '/' + OUT_DIR for d in DIRS]
else:
    raise ValueError('Length missmatch between OUT_DIR and DIRS')
out_dirs

['../../course-python-beginner/lessons/python introduction/clean']

In [31]:
for d in DIRS:
    for entry in os.scandir(d):
        if entry.is_file() and entry.name.endswith(".ipynb"):
            nb = open_nb(entry.path)
            nb = process_cells(nb)
            save_nb(d, entry.path, nb)
            nb = None
    
    print('Done Converting Files: {}'.format(entry.path))
            

Done Converting Files: ../../course-python-beginner/lessons/python introduction\0X-strings.ipynb
